# Regression over all Data

Data prep

In [14]:
from groq import Groq
import pandas as pd
import re
import os

In [15]:
df = pd.read_csv('../../data_acquisition/data/df_without_salary.csv')
df_with_salary_few_shot = pd.read_csv('../../data_acquisition/data/df_with_salary_few_shot.csv')

In [16]:
df.drop('salary', axis=1, inplace=True)

In [10]:
#df['min_salary']=None
#df['max_salary']=None

In [17]:
df

,title,company,contract_type,description,min_workload,max_workload
0,Projektleiter / Consultant e-Case.Net,Infogate AG,Festanstellung,Wer braucht Deine Verstärkung? 1991 waren wir ...,80,100
1,Senior Software Engineer 80 - 100% (m/w/d),Wey Group AG,Festanstellung,Deine Aufgaben: Als Senior Software Engineer...,80,100
2,System Engineer - Ansible,RMIT Professional Resources AG,Freelance,System Engineer Position einhergehend mit Kenn...,80,100
3,"Softwareentwickler/in mit GIS-Erfahrung, Profe...",EBP Schweiz AG,Festanstellung,Dein Beitrag in unserem Team Du entwickelst b...,60,100
4,SAP Consultant PP-PI (m/w/d),Migros-Genossenschafts-Bund,Festanstellung,"""Teamwork makes the dream work"" ist für dich k...",60,100
...,...,...,...,...,...,...
1586,System Engineer,Beltronic IT AG,Festanstellung,Wer braucht Deine Verstärkung? Wir entwickeln ...,60,100
1587,Cloud Engineer Modern Workplace,EveryWare AG,Festanstellung,Wer braucht deine Verstärkung? EveryWare ist d...,80,100
1588,System Engineer Operations,Beltronic IT AG,Festanstellung,Wer braucht Deine Verstärkung? Wir entwickeln ...,60,100
1589,Projektleiter / Senior System Engineer,Beltronic IT AG,Festanstellung,Wer braucht Deine Verstärkung? Wir entwickeln ...,80,100


Usage Setup

In [18]:
client = Groq(
    api_key="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",
)


def extract_salaries(response):
    # General regex pattern - covers most cases
    general_pattern = re.compile(r"(\d{1,3}(?:['\s]\d{3})*)\s*(?:-|und)\s*(\d{1,3}(?:['\s]\d{3})*)\s*CHF")
    # Special regex pattern - handles "CHF" before the numbers and potential markdown
    special_pattern = re.compile(r"CHF\s*(\d{1,3}(?:[',\s]\d{3})*)\s*[-–]\s*CHF\s*(\d{1,3}(?:[',\s]\d{3})*)")
    # New pattern for numbers inside parentheses with "und" and possibly no formatting
    parentheses_pattern = re.compile(r"\((\d{1,3}(?:['\s]?\d{3})*)\)\s*und\s*\((\d{1,3}(?:['\s]?\d{3})*)\)\s*CHF")
    # New pattern for a range inside parentheses with a dash
    range_parentheses_pattern = re.compile(r"\((\d{1,3}(?:['\s]\d{3})*)\s*-\s*(\d{1,3}(?:['\s]\d{3})*)\)\s*CHF")

    # Try the general pattern first
    matches = general_pattern.findall(response)
    if matches:
        min_salary, max_salary = [int(salary.replace("'", "").replace(",", "").replace(" ", "")) for salary in matches[0]]
        return min_salary, max_salary
    
    # If no matches, try the special pattern
    matches = special_pattern.findall(response)
    if matches:
        min_salary, max_salary = [int(salary.replace("'", "").replace(",", "").replace(" ", "")) for salary in matches[0]]
        return min_salary, max_salary

    # Try the new parentheses pattern
    matches = parentheses_pattern.findall(response)
    if matches:
        min_salary, max_salary = [int(salary.replace("'", "").replace(",", "").replace(" ", "")) for salary in matches[0]]
        return min_salary, max_salary

    # Try the range inside parentheses pattern
    matches = range_parentheses_pattern.findall(response)
    if matches:
        min_salary, max_salary = [int(salary.replace("'", "").replace(",", "").replace(" ", "")) for salary in matches[0]]
        return min_salary, max_salary

    return None, None

Calculation

In [34]:
# change here for new prompt and message below ------------
results_file = '../../data_acquisition/prepared_data/regression_all.csv'
min = 'min_salary'
max = 'max_salary'
answer = 'block1_answer_fewshot'
# Setup and load initial data
if os.path.exists(results_file):
    df = pd.read_csv(results_file)
else:
    df[min] = None  # Initialize columns if starting fresh
    df[max] = None
    df[answer] = None 

# change content here for new prompt and message below ------------
def generate_query(row):
    return {
        "role": "user",
        "content": f"Gib mir nur das Gehalt im Format (\d{1,3}(?:'\d{3})*) und (\d{1,3}(?:'\d{3})) CHF für die Position {row['title']} in Zürich, Schweiz. Wenn du kein exaktes Gehalt hast gib eine Schätzung an. Der angegebene Gehaltsbereich sollte dabei eine maximale Spanne von 20000 CHF nicht überschreiten."
    }

def generate_message(title, company, min_salary, max_salary, min_workload, max_workload, description):
    return {
        "role": "user",
        "content": f"Das Gehalt für die Position {title}  liegt zwischen {min_salary} und {max_salary}."
    }

def run_model(limit):
    start_index = df[min].last_valid_index() + 1 if df[max].last_valid_index() != None else 0
    example_messages = [generate_message(row['title'], row['company'], row['min_salary'], row['max_salary'], row['min_workload'], row['max_workload'], row['description']) for index, row in df_with_salary_few_shot.iterrows()]
    
    for index, row in df.iloc[start_index:start_index + limit].iterrows():
        query_message = generate_query(row)
        chat_completion = client.chat.completions.create(
            messages= example_messages + [query_message], # change message for new prompt ------------
            model="llama3-70b-8192",
            temperature=0.0
        )
        min_salary, max_salary = extract_salaries(chat_completion.choices[0].message.content)
        print(index)
        print(min_salary, max_salary)
        
        
        # Update DataFrame
        df.at[index, min] = min_salary
        df.at[index, max] = max_salary
        df.at[index, answer] = chat_completion.choices[0].message.content
        
        # Save updated DataFrame in chunks or fully, depending on your preference
        df.to_csv(results_file, index=False)

# Testing the function with a limited number of entries
run_model(10000)

1156
80000 100000
1157
90000 110000
1158
90000 110000
1159
110000 130000
1160
90000 110000
1161
110000 130000
1162
60000 80000
1163
90000 110000
1164
110000 130000
1165
90000 110000
1166
110000 130000
1167
90000 110000
1168
90000 110000
1169
110000 130000
1170
110000 130000
1171
90000 110000
1172
80000 100000
1173
80000 100000
1174
95000 115000
1175
90000 110000
1176
80000 100000
1177
90000 110000
1178
90000 110000
1179
90000 110000
1180
90000 110000
1181
60000 80000
1182
90000 110000
1183
90000 110000
1184
110000 130000
1185
80000 100000
1186
110000 130000
1187
110000 130000
1188
90000 110000
1189
110000 130000
1190
90000 110000
1191
90000 110000
1192
90000 110000
1193
90000 110000
1194
90000 110000
1195
90000 110000
1196
90000 110000
1197
90000 110000
1198
90000 110000
1199
80000 100000
1200
110000 130000
1201
40000 60000
1202
80000 100000
1203
90000 110000
1204
80000 100000
1205
90000 110000
1206
90000 110000
1207
90000 110000
1208
90000 110000
1209
110000 130000
1210
90000 110000
1

In [35]:
data_check = pd.read_csv('../../data_acquisition/prepared_data/regression_all.csv')

In [37]:
data_check.head(2000)

,title,company,contract_type,description,min_workload,max_workload,min_salary,max_salary,block1_answer_fewshot
0,Projektleiter / Consultant e-Case.Net,Infogate AG,Festanstellung,Wer braucht Deine Verstärkung? 1991 waren wir ...,80,100,90000.0,110000.0,(90'000) und (110'000) CHF
1,Senior Software Engineer 80 - 100% (m/w/d),Wey Group AG,Festanstellung,Deine Aufgaben: Als Senior Software Engineer...,80,100,110000.0,130000.0,(110000) und (130000) CHF
2,System Engineer - Ansible,RMIT Professional Resources AG,Freelance,System Engineer Position einhergehend mit Kenn...,80,100,90000.0,110000.0,(90'000) und (110'000) CHF
3,"Softwareentwickler/in mit GIS-Erfahrung, Profe...",EBP Schweiz AG,Festanstellung,Dein Beitrag in unserem Team Du entwickelst b...,60,100,90000.0,110000.0,(90'000 - 110'000) CHF
4,SAP Consultant PP-PI (m/w/d),Migros-Genossenschafts-Bund,Festanstellung,"""Teamwork makes the dream work"" ist für dich k...",60,100,90000.0,110000.0,(90'000) und (110'000) CHF
...,...,...,...,...,...,...,...,...,...
1586,System Engineer,Beltronic IT AG,Festanstellung,Wer braucht Deine Verstärkung? Wir entwickeln ...,60,100,90000.0,110000.0,(90'000) und (110'000) CHF
1587,Cloud Engineer Modern Workplace,EveryWare AG,Festanstellung,Wer braucht deine Verstärkung? EveryWare ist d...,80,100,90000.0,110000.0,(90'000) und (110'000) CHF
1588,System Engineer Operations,Beltronic IT AG,Festanstellung,Wer braucht Deine Verstärkung? Wir entwickeln ...,60,100,90000.0,110000.0,(90'000) und (110'000) CHF
1589,Projektleiter / Senior System Engineer,Beltronic IT AG,Festanstellung,Wer braucht Deine Verstärkung? Wir entwickeln ...,80,100,80000.0,100000.0,(80'000 - 100'000) CHF


Data Cleaning

In [177]:
data_check[data_check['max_salary'].isna()]

,title,company,contract_type,description,min_workload,max_workload,min_salary,max_salary,block1_answer_fewshot


In [174]:
data_check.loc[1547,"block1_answer_fewshot"]

'120000 und 140000 CHF'

In [173]:
data_check.loc[1580, 'min_salary'] = 110000
data_check.loc[1580, 'max_salary'] = 130000

In [175]:
data_check.loc[1547, 'min_salary'] = 120000
data_check.loc[1547, 'max_salary'] = 140000

In [169]:
data_check.loc[1563, 'min_salary'] = 120000
data_check.loc[1563, 'max_salary'] = 140000

In [167]:
data_check.loc[1547, 'min_salary'] = 120000
data_check.loc[1547, 'max_salary'] = 140000

In [165]:
data_check.loc[1442, 'min_salary'] = 80000
data_check.loc[1442, 'max_salary'] = 100000

In [163]:
data_check.loc[1441, 'min_salary'] = 80000
data_check.loc[1441, 'max_salary'] = 100000

In [160]:
data_check.loc[1440, 'min_salary'] = 80000
data_check.loc[1440, 'max_salary'] = 100000

In [158]:
data_check.loc[1421, 'min_salary'] = 70000
data_check.loc[1421, 'max_salary'] = 90000

In [155]:
data_check.loc[1410, 'min_salary'] = 80000
data_check.loc[1410, 'max_salary'] = 100000

In [153]:
data_check.loc[1354, 'min_salary'] = 80000
data_check.loc[1354, 'max_salary'] = 100000

In [151]:
data_check.loc[1339, 'min_salary'] = 120000
data_check.loc[1339, 'max_salary'] = 140000

In [148]:
data_check.loc[1314, 'min_salary'] = 5000
data_check.loc[1314, 'max_salary'] = 7000

In [146]:
data_check.loc[1310, 'min_salary'] = 120000
data_check.loc[1310, 'max_salary'] = 140000

In [144]:
data_check.loc[1297, 'min_salary'] = 90000
data_check.loc[1297, 'max_salary'] = 110000

In [142]:
data_check.loc[1274, 'min_salary'] = 120000
data_check.loc[1274, 'max_salary'] = 140000

In [140]:
data_check.loc[1273, 'min_salary'] = 120000
data_check.loc[1273, 'max_salary'] = 140000

In [137]:
data_check.loc[1261, 'min_salary'] = 120000
data_check.loc[1261, 'max_salary'] = 140000

In [135]:
data_check.loc[1257, 'min_salary'] = 55000
data_check.loc[1257, 'max_salary'] = 75000

In [133]:
data_check.loc[1233, 'min_salary'] = 120000
data_check.loc[1233, 'max_salary'] = 140000

In [131]:
data_check.loc[1224, 'min_salary'] = 120000
data_check.loc[1224, 'max_salary'] = 140000

In [124]:
data_check.loc[1218, 'min_salary'] = 120000
data_check.loc[1218, 'max_salary'] = 140000

In [122]:
data_check.loc[1126, 'min_salary'] = 120000
data_check.loc[1126, 'max_salary'] = 140000

In [120]:
data_check.loc[1115, 'min_salary'] = 110000
data_check.loc[1115, 'max_salary'] = 130000

In [117]:
data_check.loc[1111, 'min_salary'] = 90000
data_check.loc[1111, 'max_salary'] = 110000

In [115]:
data_check.loc[1109, 'min_salary'] = 90000
data_check.loc[1109, 'max_salary'] = 110000

In [113]:
data_check.loc[1081, 'min_salary'] = 90000
data_check.loc[1081, 'max_salary'] = 110000

In [111]:
data_check.loc[1050, 'min_salary'] = 120000
data_check.loc[1050, 'max_salary'] = 140000

In [109]:
data_check.loc[1049, 'min_salary'] = 90000
data_check.loc[1049, 'max_salary'] = 110000

In [107]:
data_check.loc[1039, 'min_salary'] = 60000
data_check.loc[1039, 'max_salary'] = 80000

In [105]:
data_check.loc[1007, 'min_salary'] = 90000
data_check.loc[1007, 'max_salary'] = 110000

In [103]:
data_check.loc[1005, 'min_salary'] = 90000
data_check.loc[1005, 'max_salary'] = 110000

In [101]:
data_check.loc[1004, 'min_salary'] = 90000
data_check.loc[1004, 'max_salary'] = 110000

In [98]:
data_check.loc[999, 'min_salary'] =120000
data_check.loc[999, 'max_salary'] = 140000

In [96]:
data_check.loc[986, 'min_salary'] = 80000
data_check.loc[986, 'max_salary'] = 100000

In [94]:
data_check.loc[984, 'min_salary'] = 80000
data_check.loc[984, 'max_salary'] = 100000

In [90]:
data_check.loc[900, 'min_salary'] = 120000
data_check.loc[900, 'max_salary'] = 140000

In [87]:
data_check.loc[893, 'min_salary'] = 90000
data_check.loc[893, 'max_salary'] = 110000

In [85]:
data_check.loc[891, 'min_salary'] = 90000
data_check.loc[891, 'max_salary'] = 110000

In [83]:
data_check.loc[879, 'min_salary'] = 70000
data_check.loc[879, 'max_salary'] = 90000

In [81]:
data_check.loc[842, 'min_salary'] = 45000
data_check.loc[842, 'max_salary'] = 65000

In [79]:
data_check.loc[816, 'min_salary'] = 120000
data_check.loc[816, 'max_salary'] = 140000

In [75]:
data_check.loc[719, 'min_salary'] = 120000
data_check.loc[719, 'max_salary'] = 140000

In [73]:
data_check.loc[713, 'min_salary'] = 120000
data_check.loc[713, 'max_salary'] = 140000

In [71]:
data_check.loc[712, 'min_salary'] = 120000
data_check.loc[712, 'max_salary'] = 140000

In [69]:
data_check.loc[529, 'min_salary'] = 120000
data_check.loc[529, 'max_salary'] = 140000

In [66]:
data_check.loc[482, 'min_salary'] = 90000
data_check.loc[482, 'max_salary'] = 110000

In [64]:
data_check.loc[480, 'min_salary'] = 90000
data_check.loc[480, 'max_salary'] = 110000

In [62]:
data_check.loc[479, 'min_salary'] = 70000
data_check.loc[479, 'max_salary'] = 90000

In [59]:
data_check.loc[478, 'min_salary'] = 90000
data_check.loc[478, 'max_salary'] = 110000

In [57]:
data_check.loc[363, 'min_salary'] = 90000
data_check.loc[363, 'max_salary'] = 110000

In [53]:
data_check.loc[250, 'min_salary'] = 90000
data_check.loc[250, 'max_salary'] = 110000

In [50]:
data_check.loc[246, 'min_salary'] = 70000
data_check.loc[246, 'max_salary'] = 90000

In [47]:
data_check.loc[219, 'min_salary'] = 45000
data_check.loc[219, 'max_salary'] = 65000

In [44]:
# Update the specified rows for block1_min_salary_fewshot
data_check.loc[168, 'min_salary'] = 90000
data_check.loc[168, 'max_salary'] = 110000

Save Data

In [187]:
data_check[data_check.isna().any(axis=1)]

,title,company,contract_type,description,min_workload,max_workload,min_salary,max_salary,block1_answer_fewshot


In [181]:
data_check.head()

,title,company,contract_type,description,min_workload,max_workload,min_salary,max_salary,block1_answer_fewshot
0,Projektleiter / Consultant e-Case.Net,Infogate AG,Festanstellung,Wer braucht Deine Verstärkung? 1991 waren wir ...,80,100,90000.0,110000.0,(90'000) und (110'000) CHF
1,Senior Software Engineer 80 - 100% (m/w/d),Wey Group AG,Festanstellung,Deine Aufgaben: Als Senior Software Engineer...,80,100,110000.0,130000.0,(110000) und (130000) CHF
2,System Engineer - Ansible,RMIT Professional Resources AG,Freelance,System Engineer Position einhergehend mit Kenn...,80,100,90000.0,110000.0,(90'000) und (110'000) CHF
3,"Softwareentwickler/in mit GIS-Erfahrung, Profe...",EBP Schweiz AG,Festanstellung,Dein Beitrag in unserem Team Du entwickelst b...,60,100,90000.0,110000.0,(90'000 - 110'000) CHF
4,SAP Consultant PP-PI (m/w/d),Migros-Genossenschafts-Bund,Festanstellung,"""Teamwork makes the dream work"" ist für dich k...",60,100,90000.0,110000.0,(90'000) und (110'000) CHF


In [188]:
data_check.to_csv('../../data_acquisition/prepared_data/regression_all_clean.csv', index=False)

safe as json

In [193]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('../../data_acquisition/prepared_data/regression_all_clean.csv', encoding='utf-8')



with open('../../data_acquisition/prepared_data/regression_all_clean.json', 'w', encoding='utf-8') as file:
    df.to_json(file, orient='records', force_ascii=False)
print("JSON file has been created successfully.")


JSON file has been created successfully.
